In [1]:
%cd /content/drive/MyDrive/Colab Notebooks/nlp/apps/name_entity_recognition

/content/drive/MyDrive/Colab Notebooks/nlp/apps/name_entity_recognition


In [6]:
path = '/content/drive/MyDrive/Colab Notebooks/nlp/apps/data/sherlock_novels.txt'
testing_path = '/content/drive/MyDrive/Colab Notebooks/nlp/apps/data/study in scarlet.txt'
ner_path = '/content/drive/MyDrive/Colab Notebooks/nlp/apps/data/ner_dataset.csv'

In [3]:
!pip install -q -U trax

     |████████████████████████████████| 471kB 5.6MB/s 
     |████████████████████████████████| 174kB 15.8MB/s 
     |████████████████████████████████| 2.6MB 16.8MB/s 
     |████████████████████████████████| 1.4MB 41.0MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 1.1MB 49.5MB/s 
     |████████████████████████████████| 348kB 45.1MB/s 
     |████████████████████████████████| 3.7MB 46.3MB/s 
     |████████████████████████████████| 890kB 48.6MB/s 
     |████████████████████████████████| 2.9MB 49.2MB/s 


In [5]:
import trax
import random
import pandas as pd
import numpy as np


# Get the data

The columns of the dataset are:

- The sentence number
- the word
- the part of speech of the word
- the tags


In [8]:
df = pd.read_csv(ner_path, encoding='ISO-8859-1')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


# Preprocess

We are going to create two files: one with the unique tags and one with the unique words 


In [22]:
def create_vocab(df, path='/content/drive/MyDrive/Colab Notebooks/nlp/apps/data/'):
    """
    Takes the ner dataset and creates a txt file
    with words that appear at least once.
    """
    # Lowercase and upercase are treated as different words
    counts = df.Word.value_counts()
    words = list(counts.index)
    with open(path + 'words.txt', 'w') as f:
        for word in words:
            f.write(word + '\n')

#create_vocab(df)

In [26]:
def create_tags(df, path='/content/drive/MyDrive/Colab Notebooks/nlp/apps/data/'):
    """
    Takes the ner dataset and create a txt file with
    the different tags
    """
    tags = list(df.Tag.value_counts().index)
    with open(path + 'tags.txt', 'w') as f:
        for tag in tags:
            if tag == 'O':
                continue
            f.write(tag + '\n')

create_tags(df)
        


Mikulski
